## 제품명 검색 결과 url 추출

In [9]:
import pandas as pd

df = pd.read_excel("전처리후 제품명만 데이터.xlsx")
df.head()

,Unnamed: 0,제품명
0,0,셀레나제마이크로그램퍼오랄액
1,1,공캡슐호
2,2,공캡슐호
3,3,가기코프캡슐
4,4,가나폴로연질캡슐


In [2]:
from bs4 import BeautifulSoup
import requests   

url = "https://terms.naver.com/medicineSearch.nhn?mode=nameSearch&query=%EC%85%80%EB%A0%88%EB%82%98%EC%A0%9C%EB%A7%88%EC%9D%B4%ED%81%AC%EB%A1%9C%EA%B7%B8%EB%9E%A8%ED%8D%BC%EC%98%A4%EB%9E%84%EC%95%A1"

req = requests.get(url)
soup = BeautifulSoup(req.content, 'html.parser')
a = soup.select('div strong a')[1]

if 'href' in a.attrs:
        link = a.attrs['href']        
    
print("https://terms.naver.com/" + link)

https://terms.naver.com//entry.nhn?docId=2147085&cid=51000&categoryId=51000


In [3]:
url = "https://terms.naver.com/medicineSearch.nhn?mode=nameSearch&query=%EC%85%80%EB%A0%88%EB%82%98%EC%A0%9C%EB%A7%88%EC%9D%B4%ED%81%AC%EB%A1%9C%EA%B7%B8%EB%9E%A8%ED%8D%BC%EC%98%A4%EB%9E%84%EC%95%A1"

req = requests.get(url)
soup = BeautifulSoup(req.content, 'html.parser')
div = soup.find_all('div', {'class' : 'subject'})

for d in div:
    a = d.select('a')[0]
    if 'href' in a.attrs:
        link = a.attrs['href']        
    
print("https://terms.naver.com/" + link)

https://terms.naver.com//entry.nhn?docId=2147085&cid=51000&categoryId=51000


In [3]:
import re

url = "https://terms.naver.com/medicineSearch.nhn?mode=nameSearch&query=가나폴로연질캡슐"

req = requests.get(url)
soup = BeautifulSoup(req.content, 'html.parser')
title = soup.select('li div strong.title')[0]

for a in title.find_all('a', href = re.compile("^(/entry.nhn)")):
    link = a.attrs['href']

print("https://terms.naver.com/" + link)

https://terms.naver.com//entry.nhn?docId=2147935&cid=51000&categoryId=51000


In [2]:
from tqdm.notebook import tqdm

links = []

for i in tqdm(range(len(df['제품명']))):
    try:
        url = "https://terms.naver.com/medicineSearch.nhn?mode=nameSearch&query={}".format(df['제품명'][i])
        req = requests.get(url)
        soup = BeautifulSoup(req.content, 'html.parser')
        title = soup.select('li div strong.title')[0]

        for a in title.find_all('a', href = re.compile("^(/entry.nhn)")):
                link = a.attrs['href']
                links.append("https://terms.naver.com/" + link)
    except:
        links.append('x')

In [20]:
urls = pd.DataFrame(data = {'links' : links }) ; urls

,links
0,https://terms.naver.com//entry.nhn?docId=21470...
1,x
2,x
3,https://terms.naver.com//entry.nhn?docId=21444...
4,https://terms.naver.com//entry.nhn?docId=21479...
...,...
11452,https://terms.naver.com//entry.nhn?docId=21267...
11453,https://terms.naver.com//entry.nhn?docId=56850...
11454,https://terms.naver.com//entry.nhn?docId=51388...
11455,https://terms.naver.com//entry.nhn?docId=21261...


In [21]:
urls.nunique()

links    10696
dtype: int64

In [22]:
urls.to_excel("medicine_url.xlsx")

## 제품 url 불러와 상세정보 크롤링

In [1]:
import pandas as pd

urls = pd.read_excel("medicine_url.xlsx")
urls.head()

,Unnamed: 0,links
0,0,https://terms.naver.com//entry.nhn?docId=21470...
1,1,x
2,2,x
3,3,https://terms.naver.com//entry.nhn?docId=21444...
4,4,https://terms.naver.com//entry.nhn?docId=21479...


In [2]:
# url 정보 없는 것은 제거

links = urls[urls['links'] != 'x'] 

In [13]:
links.to_excel("naver_medicine_url.xlsx")

In [4]:
from bs4 import BeautifulSoup
import requests   

제품 = []
식약처분류 = []
외형정보 = []
성분정보 = []
저장방법 = []
효능효과 = []
용법용량 = []
주의사항 = []

url = urls['links'][10]
req = requests.get(url)
soup = BeautifulSoup(req.content, 'html.parser')
제품.append(soup.select('h2.headword')[0].text)
식약처분류.append(soup.select('tbody td')[0].text)
외형정보.append(soup.select('p.txt')[0].text)
성분정보.append(soup.select('p.txt')[1].text)
저장방법.append(soup.select('p.txt')[2].text)
효능효과.append(soup.select('p.txt')[3].text)
용법용량.append(soup.select('p.txt')[4].text)
주의사항.append(soup.select('p.txt')[5].text)

print(제품)
print(식약처분류[0].strip('\n'))
print(식약처분류[0].split('>')[1].strip('\n'))
print(식약처분류[0].split('>')[2].strip('\n'))
print(외형정보[0].strip('\n'))
print(성분정보[0].strip('\n'))
print(저장방법[0].strip('\n'))
print(효능효과[0].strip('\n'))
print(용법용량[0].strip('\n'))
print(주의사항[0].strip('\n'))


['가스민에프정']
                                        개개의 기관계용 의약품 > 소화기관용약 > 제산제

                                
 소화기관용약 
 제산제

                                
· 성상 : 흰색의 장방형 정제
건조수산화알루미늄겔 450mg, 탄산마그네슘 288mg
밀폐용기, 실온(1~30℃)보관
위산과다, 속쓰림, 위부불쾌감, 위부팽만감, 식체(위체), 구역, 구토, 위통, 신트림
성인 1회 2정, 1일 3회 식후에 복용
1. 다음 환자에는 투여하지 말 것.투석요법을 받고 있는 환자2. 다음 환자에는 신중히 투여할 것. 1) 신장애 환자 2) 다른 약물을 투여받고 있는 환자 3. 부작용 이 약을 투여하는 동안 변비 또는 설사의 증상이 나타날 수 있다. 4. 일반적 주의 1) 정해진 용법ㆍ용량을 잘 지킨다. 2) 2주 정도 투여하여도 증상의 개선이 없을 경우에는 투여를 중지하고 약사 또는 의사와 상의한다. 3) 장기연용을 하지 않는다.4) 신장병의 병력이 있는 환자는 복용전에 의사 또는 약사와 상의한다.5. 소아에 대한 투여 이 약은 7세 이하의 영ㆍ유아에게 투여하지 않는다. 6. 저장상의 주의 1) 소아의 손이 닿지 않는 곳에 보관한다. 2) 직사일광을 피하고 되도록이면 서늘한 곳에 보관한다. 3) 오용을 막고 품질의 보존을 위하여 다른 용기에 바꾸어 넣지 않는다.


In [5]:
from tqdm.notebook import tqdm
import time
import re

result_list = []

for link in tqdm((links['links'])):
    req = requests.get(link)
    soup = BeautifulSoup(req.text, 'html.parser')
   
    try:
        제품 = soup.select('h2.headword')[0].text
        식약처분류1 = re.sub("[\r\n\t]", "", str(soup.select('tbody td')[0].text.strip().split('>')[0]))
        식약처분류2 = re.sub("[\r\n\t]", "", str(soup.select('tbody td')[0].text.strip().split('>')[1]))
        식약처분류3 = re.sub("[\r\n\t]", "", str(soup.select('tbody td')[0].text.strip().split('>')[2]))
        외형정보 =  re.sub("[\r\n\t]", "", str(soup.select('p.txt')[0].text.strip()))
        성분정보 =  re.sub("[\r\n\t]", "", str(soup.select('p.txt')[1].text.strip()))
        저장방법 =  re.sub("[\r\n\t]", "", str(soup.select('p.txt')[2].text.strip()))
        효능효과 =  re.sub("[\r\n\t]", "", str(soup.select('p.txt')[3].text.strip()))
        용법용량 =  re.sub("[\r\n\t]", "", str(soup.select('p.txt')[4].text.strip()))
        주의사항 =  re.sub("[\r\n\t]", "", str(soup.select('p.txt')[5].text.strip()))
    except:
        제품 = 'x'
        식약처분류1 = 'x'
        식약처분류2 = 'x'
        식약처분류3 = 'x'
        외형정보 = 'x'
        성분정보 = 'x'
        저장방법 = 'x'
        효능효과 = 'x'
        용법용량 = 'x'
        주의사항 = 'x'
        
    result = 제품, 식약처분류1, 식약처분류2, 식약처분류3, 외형정보, 성분정보, 저장방법, 효능효과, 용법용량, 주의사항
    result_list.append(result)
        
    time.sleep(0.5)

In [6]:
medicine_data = pd.DataFrame(result_list, columns = ['제품', '식약처분류1', '식약처분류2', '식약처분류3', '외형정보', '성분정보', '저장방법', '효능효과', '용법용량', '주의사항'])

In [7]:
medicine_data.head()

,제품,식약처분류1,식약처분류2,식약처분류3,외형정보,성분정보,저장방법,효능효과,용법용량,주의사항
0,1.셀레나제100μg퍼오랄액 2.셀레나제티퍼오랄액,대사성 의약품,자양강장변질제,무기질제제,· 성상 : 흰색 플라스틱 용기에 든 무색 투명한 액제,"아셀렌산나트륨오수화물 0.332mg/2mL, 아셀렌산나트륨오수화물 1.66mg/10mL","기밀용기, 상온(15~25℃)","[허가사항변경(2012년 재평가) 의약품관리총괄과-9301, 2013.12.30]영...","셀레늄으로서 1일 100μg(2mL)을 복용하며, 1일 셀레늄으로 200μg(4mL...","[허가사항변경(2012년 재평가) 의약품관리총괄과-9301, 2013.12.30]1..."
1,가기코프캡슐,개개의 기관계용 의약품,호흡기관용약,진해거담제,· 성상 : 이 약은 백색의 내용물이 든 상.하 미황색으로된 경질캅셀제이다.,"DL-메틸에페드린염산염 12.5mg, 구아야콜설폰산칼륨 46.67mg, 노스카핀 1...","기밀용기, 실온보관(1-30℃)","기침, 가래","성인 : 1회 2캅셀, 1일 3회 식후복용",1. 다음 환자에는 투여하지 말 것. 임부 또는 임신하고 있을 가능성이 있는 부...
2,가나폴로연질캡슐,대사성 의약품,비타민제,"혼합비타민제(비타민A, D 혼합제를 제외)",· 성상 : 황색의 내용물을 함유하는 적갈색의 연질캡슐제· 제형 : 연질캡슐· 모양...,"니코틴산아미드 20mg, 리보플라빈 3mg, 아스코르브산 75mg, 콜레칼시페롤유 ...","기밀용기, 실온(1~30℃)보관","1. 다음 경우의 비타민 A, B1, B2, C, D의 보급- 육체피로, 임신ㆍ수유...","성인 1일 2회, 1회 1캡슐 복용",1. 경고 임부에 비타민A (레티놀)를 1일 5000 I.U. 이상 투여하는 경우에...
3,가네쿨연질캡슐,대사성 의약품,기타의 대사성 의약품,따로 분류되지 않는 대사성 의약품,· 성상 : 유백색의 점조성 내용물이 든 갈색의 연질캡슐제· 제형 : 연질캡슐· 모...,"L-시스틴 250mg, 콜린타르타르산염 250mg","기밀용기, 실온(1～30℃)보관",간기능 저하로 인한 소화불량의 보조치료,성인 : 1일 1~2캡슐을 식전 또는 소화불량시 복용하며 1일 4캡슐을 초과하지 않는다.,1. 다음과 같은 사람은 이 약을 복용하지 말 것.- 중증의 간세포기능부전이나 담도...
4,가네탑골드연질캡슐,대사성 의약품,기타의 대사성 의약품,간장질환용제,· 성상 : 황갈색의 내용물을 함유하는 암갈색의 장방형 연질캡슐제· 제형 : 연질캡...,"니코틴산아미드 12mg, 시아노코발라민 1.2μg, 밀크시슬열매건조엑스 200mg,...","차광기밀용기, 실온보관(1～30℃)","[허가사항변경(2016년 재평가), 의약품안전평가과-7412, 2017.12.21....","[허가사항변경(2016년 재평가), 의약품안전평가과-7412, 2017.12.21....","[허가사항변경(2016년 재평가), 의약품안전평가과-7412, 2017.12.21...."


In [8]:
medicine_data.to_excel("naver_medicine_description.xlsx")